In [142]:
%run ./links.ipynb
import pandas as pd
import requests


In [143]:
from datetime import datetime, timedelta
def time_diff(start_time_str, end_time_str="08:10:00"):
    start_time = datetime.strptime(start_time_str, '%H:%M:%S')
    end_time = datetime.strptime(end_time_str, '%H:%M:%S')
    time_delta = end_time - start_time
    time_diff_seconds = int(time_delta.total_seconds())
    return time_diff_seconds

In [144]:
from pandas.tseries.offsets import BDay
from calendar import monthrange

def generate_business_days_in_month(date_str: str):
    # Convert input date string to pandas datetime
    date = pd.to_datetime(date_str)

    # Extract year and month from the input date
    year = date.year
    month = date.month

    # Determine the last valid day of the month
    last_day = monthrange(year, month)[1]

    # Update the end date of the date range to the last valid day of the month
    date_range = pd.date_range(start=f"{year}-{month}-01", end=f"{year}-{month}-{last_day}", freq='B')

    # Filter the dates to get only business days
    business_days = date_range[date_range.isin(pd.date_range(date_range[0], date_range[-1], freq=BDay()))]

    # Convert the resulting pandas DatetimeIndex to a list of strings in YYYY-mm-dd format
    business_days_list = business_days.strftime('%Y-%m-%d').tolist()

    return business_days_list

# Example usage
date_str = "2023-01-01"
business_days = generate_business_days_in_month(date_str)
len(business_days)


22

In [145]:
link_to_emp_logs = links_ns.logbook.prange+'6/2023-04-01/2023-04-30'
emp_logs = requests.get(link_to_emp_logs).json()
emp_logs = pd.json_normalize(emp_logs)
select_cols = ["logbookId","logDate","checkinTime"]
emp_logs = emp_logs[select_cols]

In [146]:

bdays.head(2)
bdays = pd.DataFrame(generate_business_days_in_month("2023-04-30"), columns=["bdays"])

In [147]:
df = pd.merge(bdays, emp_logs, how="left", left_on="bdays", right_on="logDate")
df = df[["bdays","logbookId","checkinTime"]]
df.logbookId.fillna(0, inplace=True)
df.checkinTime.fillna("23:59:59", inplace=True)
df.head(2)

,bdays,logbookId,checkinTime
0,2023-04-03,1105.0,07:55:00
1,2023-04-04,1106.0,08:16:00


In [148]:
df["isLate"] = df.checkinTime.map(time_diff) < 0
df["isPresent"] = df.logbookId > 0
df["isLate"] = df.isLate * df.isPresent
df.rename(columns={"bdays":"logDate","checkinTime":"checkIn"}, inplace=True)
#df = df[["logDate","checkIn","isPresent","isLate"]]

In [149]:
df

,logDate,logbookId,checkIn,isLate,isPresent
0,2023-04-03,1105.0,07:55:00,False,True
1,2023-04-04,1106.0,08:16:00,True,True
2,2023-04-05,1107.0,08:25:00,True,True
3,2023-04-06,1108.0,08:22:00,True,True
4,2023-04-07,1109.0,07:50:00,False,True
5,2023-04-10,0.0,23:59:59,False,False
6,2023-04-11,1110.0,08:02:00,False,True
7,2023-04-12,0.0,23:59:59,False,False
8,2023-04-13,0.0,23:59:59,False,False
9,2023-04-14,0.0,23:59:59,False,False


In [150]:
df.isLate.value_counts()

isLate
False    17
True      3
Name: count, dtype: int64

In [151]:
df.isPresent.value_counts()

isPresent
True     11
False     9
Name: count, dtype: int64

# Display data monthly

In [153]:
df.logDate.month

AttributeError: 'Series' object has no attribute 'month'